In [1]:
import pandas as pd
import numpy as np
import chardet
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
filepath = '/Users/andrewlipschultz/Dropbox/My Mac (Andrew’s MacBook Pro)/Documents/GitHub/cs224N_final_project/nytcrosswords.csv'
filepath = 'nytcrosswords.csv'
with open(filepath, 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']


In [2]:
df = pd.read_csv(filepath, encoding=encoding)
print(df.head())

relevant_cols = ['Clue', 'Word']
for col in relevant_cols:
    text_data = df[col].apply(str).tolist()
    training_text = "\n".join(text_data)


         Date     Word                                 Clue
0  10/31/2021      PAT  Action done while saying "Good dog"
1  10/31/2021  RASCALS                      Mischief-makers
2  10/31/2021      PEN          It might click for a writer
3  10/31/2021      SEP                             Fall mo.
4  10/31/2021      ECO                Kind to Mother Nature


In [3]:
with open('training_data.txt', 'w', encoding='utf-8') as file:
    file.write(training_text)

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='training_data.txt',
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

from transformers import pipeline

/Users/andrewlipschultz/miniconda3/envs/cs224n/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text
text_prompt = ""
generated_text = generator(text_prompt, max_length=100, num_return_sequences=1)
print(generated_text[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


There are currently 17,458 private web sites (listed and populated by users over the world) that use Google Analytics to conduct marketing and advertising related research and analyze customer information to optimize content.

If you're interested in finding out more – get to know others before jumping forward to the next phase of your research by clicking on the link!


To see the full list of web sites by user, click here!


In [18]:
clues = []
answers = []
lengths = []
with open('output.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip().strip('"')
        parts = line.rsplit(',', 2)
        if len(parts) == 3:
            clue = parts[0].strip().strip('"')
            answer = parts[1].strip().strip('"')
            length = int(parts[2].strip())
            clues.append(clue)
            answers.append(answer)
            lengths.append(lengths)

            

In [ ]:
for i in range(5):
    length = lengths[0]
    clue = clues[0]
    prompt = f"Length of the word is: {length} Clue: {clue} Answer:"
    response = generator(prompt, max_length=50, num_return_sequences=1)
    full_response = response[0]['generated_text']
    answer = full_response.split("Answer:")[1].strip().split('\n')[0].strip()
    print(answer)